In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# -------------------------------
# Define your folders
palette_folder = "../Produced Images/Fingernail Palette Yolo Cropped"
original_folder = "../Dataset Used/Right Fingernail_files/Images_right_fingernail"
output_folder = "../Produced Images/Fingernail Transformed by 3x4"

# Create the output folder if it does not exist.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# -------------------------------
# Step 2: Define Reference Colors (B, G, R)
# -------------------------------
reference_colors = {
    "black": (0,   0,   0),
    "white": (255, 255, 255),
    "red":   (0,   0,   255),
    "green": (0,   255, 0),
    "blue":  (255, 0,   0)
}

# -------------------------------
# Step 3: Process Each Palette Image
# -------------------------------
for filename in os.listdir(palette_folder):
    # Process only image files
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    palette_path = os.path.join(palette_folder, filename)
    original_path = os.path.join(original_folder, filename)
    
    # Check if corresponding original image exists
    if not os.path.exists(original_path):
        print(f"Original image not found for {filename}. Skipping...")
        continue

    # Load the palette image
    palette_img = cv2.imread(palette_path)
    if palette_img is None:
        print(f"Error loading palette image: {filename}. Skipping...")
        continue

    # Convert palette image to grayscale and reduce noise with a median blur
    gray = cv2.cvtColor(palette_img, cv2.COLOR_BGR2GRAY)
    gray_blurred = cv2.medianBlur(gray, 5)

    # -------------------------------
    # Step 4: Detect Circles using HoughCircles
    # -------------------------------
    circles = cv2.HoughCircles(
        gray_blurred,
        cv2.HOUGH_GRADIENT,
       dp=1,
        minDist=30,
        param1=50,
        param2=30,
        minRadius=40,
        maxRadius=100
    )

    # Continue only if exactly 5 circles are detected
    if circles is not None:
        circles = np.uint16(np.around(circles[0]))
        if len(circles) != 5:
            print(f"{filename}: Detected {len(circles)} circles (expected 5). Skipping...")
            continue
    else:
        print(f"{filename}: No circles detected. Skipping...")
        continue

    print(f"{filename}: Detected circles (x, y, radius):")
    print(circles)

    # -------------------------------
    # Step 5: Measure Colors in Each Detected Circle
    # -------------------------------
    measured_colors = []  # Will hold average (B, G, R) colors for each circle

    for (x, y, r) in circles:
        mask = np.zeros_like(gray, dtype=np.uint8)
        cv2.circle(mask, (x, y), r, 255, -1)  # White circle on black background
        mean_color = cv2.mean(palette_img, mask=mask)  # Returns (B, G, R, alpha)
        measured_colors.append((mean_color[0], mean_color[1], mean_color[2]))

    print(f"{filename}: Measured colors (B, G, R):")
    for color in measured_colors:
        print(color)

    # -------------------------------
    # Step 6: Match Measured Colors to Reference Colors
    # -------------------------------
    matched_pairs = []  # List to hold (measured_color, reference_color)
    used_refs = set()

    for mc in measured_colors:
        mc_arr = np.array(mc, dtype=np.float32)
        best_label = None
        best_dist = float("inf")
        
        for label, ref_color in reference_colors.items():
            if label in used_refs:
                continue  # Use each reference color only once
            ref_arr = np.array(ref_color, dtype=np.float32)
            dist = np.linalg.norm(mc_arr - ref_arr)
            if dist < best_dist:
                best_dist = dist
                best_label = label
                best_ref = ref_color
                
        used_refs.add(best_label)
        matched_pairs.append((mc, best_ref))

    print(f"{filename}: Matched Measured Colors to Reference Colors:")
    for meas, ref in matched_pairs:
        print("Measured:", meas, "-> Reference:", ref)

    # -------------------------------
    # Step 7: Compute 3x4 Affine Color Correction Matrix
    # -------------------------------
    # Create arrays from the matched pairs. They will have shape (5, 3).
    meas_arr = np.array([pair[0] for pair in matched_pairs], dtype=np.float32)
    ref_arr  = np.array([pair[1] for pair in matched_pairs], dtype=np.float32)

    # Extend the measured array with a column of ones to account for an offset (affine transform)
    ones = np.ones((meas_arr.shape[0], 1), dtype=np.float32)
    meas_arr_extended = np.hstack([meas_arr, ones])  # Shape becomes (5, 4)

    # Solve for A in: meas_arr_extended * A = ref_arr, where A is a (4, 3) matrix.
    A, residuals, rank, s = np.linalg.lstsq(meas_arr_extended, ref_arr, rcond=None)

    # To apply the transform as: corrected_color = M_affine @ [measured_color; 1],
    # we need M_affine as a (3, 4) matrix. (Transpose A.)
    M_affine = A.T

    print(f"{filename}: Computed Color Correction Affine Matrix (3x4):")
    print(M_affine)

    # -------------------------------
    # Step 8: Apply the Affine Color Correction to the Original Image
    # -------------------------------
    # Load the corresponding original image
    original_img = cv2.imread(original_path)
    if original_img is None:
        print(f"Error loading original image: {filename}. Skipping...")
        continue

    # Reshape the original image to a 2D array of pixels (each pixel is a 3-element vector)
    h, w, c = original_img.shape
    pixels = original_img.reshape((-1, 3)).astype(np.float32)

    # Extend the pixel array by appending a column of ones (for the offset)
    ones_pixels = np.ones((pixels.shape[0], 1), dtype=np.float32)
    pixels_extended = np.hstack([pixels, ones_pixels])  # Shape becomes (N, 4)

    # Apply the affine transformation: each pixel is transformed by M_affine
    corrected_pixels = np.dot(pixels_extended, M_affine.T)

    # Clamp pixel values to the valid range [0, 255]
    corrected_pixels = np.clip(corrected_pixels, 0, 255)

    # Reshape back to the original image dimensions and convert to uint8
    corrected_img = corrected_pixels.reshape((h, w, 3)).astype(np.uint8)

    # -------------------------------
    # Step 9: Save the Transformed Image
    # -------------------------------
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, corrected_img)
    print(f"{filename}: Transformed image saved to {output_path}")

print("Processing completed for all images.")


1710068253661.jpg: Detected circles (x, y, radius):
[[ 68  64  61]
 [174 122  62]
 [ 70 306  62]
 [ 68 184  60]
 [176 248  60]]
1710068253661.jpg: Measured colors (B, G, R):
(122.24193134149473, 18.387552435579146, 26.1716462631624)
(251.24774065168725, 202.69347483624907, 205.94967249813448)
(64.50766934748363, 46.035154630627645, 188.94859464389353)
(79.03374966781823, 131.24067676499246, 72.53778013995925)
(33.546195411462485, 18.994862255292762, 21.636726016476214)
1710068253661.jpg: Matched Measured Colors to Reference Colors:
Measured: (122.24193134149473, 18.387552435579146, 26.1716462631624) -> Reference: (0, 0, 0)
Measured: (251.24774065168725, 202.69347483624907, 205.94967249813448) -> Reference: (255, 255, 255)
Measured: (64.50766934748363, 46.035154630627645, 188.94859464389353) -> Reference: (0, 0, 255)
Measured: (79.03374966781823, 131.24067676499246, 72.53778013995925) -> Reference: (0, 255, 0)
Measured: (33.546195411462485, 18.994862255292762, 21.636726016476214) -> Ref